# 1. Problem 1 

## b (ii)



In [ ]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import  accuracy_score 

In [ ]:
data, target = load_breast_cancer(return_X_y=True)
X, y = data, target.reshape(-1,1) 
y = np.where(y<1, -1, 1)
std = StandardScaler()
X = std.fit_transform(X)

In [ ]:
X = np.c_[X, np.ones((X.shape[0], 1))]

In [ ]:
objective_values = [] 
epsilon = 1e-5
rng = np.random.default_rng()
W = np.zeros((X.shape[1],1) )
C = 3

In [ ]:
def loss_function(z): 
  return np.where( z >= 1, 0 , (1-z)**2)

def objective_function(W ,C=C, X=X, y=y):
  return (0.5 * np.linalg.norm(W)**2 + C * (np.ones((1, X.shape[0])).dot(loss_function( y * (X @ W)))))[0][0]

def loss_gradient(z): 
  return np.where(z >= 1, 0, 2*(z-1))

def objective_function_gradient(W , C=C, X=X, y=y): 
  return W + C * (loss_gradient(y * (X @ W)).T.dot( X * y)).T 


In [ ]:
k = 0 
current_gradient = objective_function_gradient(W) 

while np.linalg.norm(current_gradient) > epsilon: 
  acc = accuracy_score(y, np.where(X@W > 0, 1, -1)) 
  if k % 50 == 0:
    print(f'{k} iter acc {acc}')
  objective_value = objective_function(W)

  objective_values.append(objective_value)

  rho, t, c = rng.random(3)

  while  objective_function(W - t* current_gradient) - objective_value  > -c*t* np.linalg.norm(current_gradient)**2:
    t *= rho 

  W   -=  t * current_gradient

  current_gradient = objective_function_gradient(W) 

  k+=1 
  

0 iter acc 0.37258347978910367
50 iter acc 0.9876977152899824
100 iter acc 0.9894551845342706
150 iter acc 0.9894551845342706
200 iter acc 0.9894551845342706
250 iter acc 0.9894551845342706
300 iter acc 0.9894551845342706
350 iter acc 0.9894551845342706
400 iter acc 0.9894551845342706
450 iter acc 0.9894551845342706
500 iter acc 0.9894551845342706
550 iter acc 0.9894551845342706
600 iter acc 0.9894551845342706
650 iter acc 0.9894551845342706
700 iter acc 0.9894551845342706
750 iter acc 0.9894551845342706
800 iter acc 0.9894551845342706
850 iter acc 0.9894551845342706
900 iter acc 0.9894551845342706
950 iter acc 0.9894551845342706
1000 iter acc 0.9894551845342706
1050 iter acc 0.9894551845342706
1100 iter acc 0.9894551845342706
1150 iter acc 0.9894551845342706
1200 iter acc 0.9894551845342706
1250 iter acc 0.9894551845342706
1300 iter acc 0.9894551845342706
1350 iter acc 0.9894551845342706
1400 iter acc 0.9894551845342706
1450 iter acc 0.9894551845342706
1500 iter acc 0.9894551845342706

In [ ]:
np.save('model.npy', W)

In [ ]:
accuracy_score(y, np.where(X@W > 0, 1, -1)) 

0.0

## b (iii)


In [ ]:
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV 

In [ ]:
svc = SVC(kernel='linear', random_state=42)

In [ ]:
config = {'C': list(range(1, 200, 1))}
clf = GridSearchCV(svc, config, verbose=1, n_jobs=-1, refit=True)
clf.fit(X, y.ravel())

Fitting 5 folds for each of 199 candidates, totalling 995 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 390 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 992 out of 995 | elapsed:   22.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 995 out of 995 | elapsed:   22.8s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='linear', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                               15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
                               27, 28, 29, 30, ...]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [ ]:
print(f'Best parameters: {clf.best_params_}')
best_estiamtor = clf.best_estimator_ 

print(f'Best Accuracy is {accuracy_score(y, best_estimator.predict(X))}')

Best parameters: {'C': 3}
Best Accuracy is 0.9912126537785588


## c (i)

In [ ]:
data, target = load_breast_cancer(return_X_y=True)
X, y = data, target.reshape(-1,1) 
y = np.where(y<1, -1, 1)
std = StandardScaler()
X = std.fit_transform(X)

In [ ]:
X.shape 
y.shape 

(569, 1)

In [ ]:
class Gram: 
  def __init__(self, data, kernel): 

    self.data = data
    self.kernel = kernel 

    self.N = self.data.shape[0]
    self.G = np.zeros((self.N, self.N))

    self.creat() 
    
  def creat(self): 
    for first_id in range(self.N): 
      for sec_id in range(self.N): 
        self.G[first_id, sec_id] = self.kernel(self.data[first_id,:],  self.data[sec_id,:])

  def __getitem__(self, ids):
    return self.G[ids]

In [ ]:
def loss_function(z): 
  return np.where( z >= 1, 0 , (1-z)**2)

def objective_function(G, y, alpha, C=C): 
  return (1/2 * alpha.T.dot(G).dot(alpha) + C * np.ones((1, G.shape[0])).dot(loss_function(y * (G @ alpha) )))[0][0]

def loss_gradient(z): 
  return np.where(z >= 1, 0, 2*(z-1))

def objective_function_gradient(G, y, alpha, C=C):
  return G.dot(alpha) +  C * (loss_gradient(y * (G @ alpha)).T.dot(G * y)).T

In [ ]:
def train(X, y, kernel, epsilon = 1e-2, C = 1.6): 
  G = Gram(X, kernel)
  Gram_matrix = G.G 


  objective_values = [] 
  rng = np.random.default_rng()
  alpha = np.zeros(((X.shape[0], 1)))

  k = 0 
  
  current_gradient = objective_function_gradient(Gram_matrix, y, alpha) 

  while np.linalg.norm(current_gradient) > epsilon: 
    
    objective_value = objective_function(Gram_matrix, y, alpha)

    objective_values.append(objective_value)

    rho, t, c = rng.random(3)

    while  objective_function(Gram_matrix, y, alpha - t * current_gradient) - objective_value  > -c*t* np.linalg.norm(current_gradient)**2:
      t *= rho 

    alpha -= t * current_gradient

    current_gradient = objective_function_gradient(Gram_matrix, y, alpha) 

    k+=1 
    acc = accuracy_score(y, np.where(alpha.T.dot(Gram_matrix).reshape(y.shape[0], -1) > 0, 1, -1)) 

    if k % 500 == 0:
      print(f'{k} iter acc {acc}')
      
  print(f'The Final accuracy is {acc}')
  return alpha


In [ ]:
from functools import partial
rbf_kernel = lambda x1, x2, gamma=0.2: np.exp((-1/2*gamma**2)*np.linalg.norm(x1 - x2)**2)
poly_kernel = lambda x1, x2, degree=3, c=1: (x1.T.dot(x2) + c)** degree  

In [ ]:
alpha1  = train(X, y, partial(rbf_kernel, gamma=0.2), epsilon=1, C=1.6)

1000 iter acc 0.9894551845342706


In [ ]:
alpha2  = train(X, y, partial(poly_kernel, degree=3, c=1,), epsilon=1, C =1)

500 iter acc 0.984182776801406
1000 iter acc 0.984182776801406
1500 iter acc 0.984182776801406
2000 iter acc 0.984182776801406
2500 iter acc 0.9859402460456942
3000 iter acc 0.9859402460456942
3500 iter acc 0.9859402460456942
4000 iter acc 0.9859402460456942
4500 iter acc 0.9876977152899824
5000 iter acc 0.9894551845342706
5500 iter acc 0.9894551845342706
6000 iter acc 0.9894551845342706
6500 iter acc 0.9912126537785588
7000 iter acc 0.9947275922671354
7500 iter acc 0.9947275922671354
8000 iter acc 0.9947275922671354
8500 iter acc 0.9947275922671354
9000 iter acc 0.9947275922671354
9500 iter acc 0.9947275922671354
10000 iter acc 0.9947275922671354
10500 iter acc 0.9947275922671354
11000 iter acc 0.9947275922671354
11500 iter acc 0.9947275922671354
12000 iter acc 0.9947275922671354
12500 iter acc 0.9947275922671354
13000 iter acc 0.9947275922671354
13500 iter acc 0.9947275922671354
14000 iter acc 0.9947275922671354
14500 iter acc 0.9947275922671354
15000 iter acc 0.9947275922671354
1550

## c (ii)

### Polynomial Kernels 

In [ ]:
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
search = RandomizedSearchCV

In [ ]:
data, target = load_breast_cancer(return_X_y=True)
X, y = data, target.reshape(-1,1) 
y = np.where(y<1, -1, 1)
std = StandardScaler()
X = std.fit_transform(X)

In [ ]:
svc  = SVC(random_state=42)
config = {'kernel': ['rbf'], 'gamma': np.linspace(0, 20, 100), 'C': np.linspace(0, 20, 100)}
          
clf = GridSearchCV(svc, config, n_jobs=-1, verbose= 1)

clf.fit(X, y.ravel())

print(f'Best parameters: {clf.best_params_}')
best_estimator = clf.best_estimator_ 

print(f'Best Accuracy is {accuracy_score(y, best_estimator.predict(X))}')

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 1066 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 1646 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 2646 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 5846 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 8046 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 10646 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 13646 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 17046 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 20846 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 25046 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 29646 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 34646 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 40046 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 45846 

Best parameters: {'C': 1.6161616161616161, 'gamma': 0.20202020202020202, 'kernel': 'rbf'}
Best Accuracy is 0.9964850615114236


[Parallel(n_jobs=-1)]: Done 50000 out of 50000 | elapsed: 15.2min finished


In [ ]:
svc  = SVC(random_state=42)
config =  {'kernel':['poly'], 'degree': list(range(1, 20)), 'coef0':list(range(0, 20)), 'C': np.linspace(0, 20, 20)}
          
clf = GridSearchCV(svc, config, n_jobs=-1, verbose= 1)

clf.fit(X, y.ravel())

print(f'Best parameters: {clf.best_params_}')
best_estimator = clf.best_estimator_ 

print(f'Best Accuracy is {accuracy_score(y, best_estimator.predict(X))}')

Fitting 5 folds for each of 7600 candidates, totalling 38000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 1030 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 10630 tasks      | elapsed:   39.9s
[Parallel(n_jobs=-1)]: Done 26630 tasks      | elapsed:  1.8min


Best parameters: {'C': 1.0526315789473684, 'coef0': 1, 'degree': 3, 'kernel': 'poly'}
Best Accuracy is 0.9876977152899824


[Parallel(n_jobs=-1)]: Done 38000 out of 38000 | elapsed:  2.5min finished
